In [ ]:
import pandas as pd
#import MetaTrader5 as mt5
import talib as ta
from bb_setup_1 import setup_2,main_indicators
import warnings
warnings.filterwarnings("ignore")

In [ ]:
ativo = 'wdo_1_full'
colnames = ['Data','Open','High','Low','Close','Vol']
df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, skiprows=1)
df = df[df['Data'] > '2022-06-01']

In [ ]:
pontos = 3
rate_stop = 1
rate_tp = 2
spread = 5

df_total = None

for timeframe in [1,5,15]:
    for var_bb in [2]:
        for pontos in [150,200,250,300,350]:
            for rate_tp in [1.5,2,2.5,3]:
                
                ativo = f'win_{timeframe}_full'
                colnames = ['Data','Open','High','Low','Close','Vol']
                df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, skiprows=1)
                df = df[df['Data'] > '2022-06-01']

                df_acao = setup_2(df,var_bb = var_bb,time_period = 20,pontos = pontos,rate_stop = 1,rate_tp = rate_tp)

                qtd_operacoes = df_acao.shape[0]
                taxa_acerto = df_acao['resultado_binario'].sum()/qtd_operacoes
                operacoes_certas = qtd_operacoes * taxa_acerto
                operacoes_erradas = qtd_operacoes - operacoes_certas
                pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

                pontos_liquido = pontos_bruto - spread * qtd_operacoes


                df_new = pd.DataFrame([[ativo[:-5],var_bb,rate_tp,rate_stop,pontos,qtd_operacoes,taxa_acerto,pontos_bruto,pontos_liquido]],
                            columns = ['ativo','var_bb','rate_tp','rate_stop','pontos','qtd_operacoes','taxa_acerto','pontos_bruto','pontos_liquido'])

                df_total = pd.concat([df_total,df_new])

                print(df_total.to_string())

In [ ]:
df_total = pd.read_csv('result_bb_setup_2_win.csv')
#df_total.to_csv('result_bb_setup_2_wdo.csv')

In [ ]:
df_1 = df_total[df_total['ativo'] == 'win_1']
df_5 = df_total[df_total['ativo'] == 'win_5']
df_15 = df_total[df_total['ativo'] == 'win_15']

In [ ]:
df_1.sort_values(by = 'pontos_liquido', ascending = False).head(10)

In [ ]:
df_5.sort_values(by = 'pontos_liquido', ascending = False).head(15)

In [ ]:
df_15.sort_values(by = 'pontos_liquido', ascending = False).head(20)

In [ ]:
timeframe = 15
for index,row in df_15.sort_values(by = 'pontos_liquido', ascending = False).head(5).iterrows():

    pontos = row['pontos']
    rate_tp = row['rate_tp']
    var_bb = row['var_bb']
    rate_stop = 1
    spread = 0.5

    ativo = f'win_{timeframe}_full'
    colnames = ['Data','Open','High','Low','Close','Vol']
    df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, skiprows=1)
    df = df[df['Data'] > '2022-01-01']

    df_acao = setup_2(df,var_bb = var_bb,time_period = 20,pontos = pontos,rate_stop = 1,rate_tp = rate_tp)

    qtd_operacoes = df_acao.shape[0]
    taxa_acerto = df_acao['resultado_binario'].sum()/qtd_operacoes
    operacoes_certas = qtd_operacoes * taxa_acerto
    operacoes_erradas = qtd_operacoes - operacoes_certas
    pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

    pontos_liquido = pontos_bruto - spread * qtd_operacoes


    print(f'Timeframe: {timeframe} --pontos: {pontos} --- rate_tp: {rate_tp} -- var_bb: {var_bb}')

    print(f'Pontos_liquido: {pontos_liquido} -- Total_operacoes: {qtd_operacoes}')

    df_acao['Hora'] = df_acao['Data'].apply(lambda x: x[11:])
    df_acao['Hora_h'] = df_acao['Hora'].apply(lambda x: x[:2])
    df_acao['Mes'] = df_acao['Data'].apply(lambda x: x[5:7])

    df_new_mes = df_acao[['Mes','resultado_binario']]
    df_new_hora = df_acao[['Hora_h','resultado_binario']]

    def calc_pontos(x):
        qtd_operacoes = x['qtd_operacoes']
        taxa_acerto = x['taxa_acerto']
        operacoes_certas = qtd_operacoes * taxa_acerto
        operacoes_erradas = qtd_operacoes - operacoes_certas
        pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

        pontos_liquido = pontos_bruto - spread * qtd_operacoes

        return pontos_liquido

    print('Agrupado por Hora')
    df_new_hora = df_new_hora.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

    df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
    print(df_new_hora)

    print('Agrupado por Mes')
    df_new_mes = df_new_mes.groupby(['Mes'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

    df_new_mes['pontos_liquido'] = df_new_mes.apply(calc_pontos, axis = 1)
    print(df_new_mes)
    print('')


### Modelo

In [ ]:
import pandas as pd
#import MetaTrader5 as mt5
import talib as ta
from bb_setup_1 import setup_2,main_indicators
import warnings
warnings.filterwarnings("ignore")

In [ ]:
timeframe = 5
pontos = 300
rate_tp = 2
var_bb = 2

ativo = f'win_{timeframe}_full'
colnames = ['Data','Open','High','Low','Close','Vol']
df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, skiprows=1)
df = df[df['Data'] > '2022-06-01']

df_acao = setup_2(df,var_bb = var_bb,time_period = 20,pontos = pontos,rate_stop = 1,rate_tp = rate_tp)
df_acao['dist_bandas'] = df_acao['BB_up'] - df_acao['BB_low']

df_acao = main_indicators(df_acao)

In [ ]:
df_acao[['Data','Close','acao', 'stop','tp','resultado_binario']].tail(10)

In [ ]:
df_acao['resultado_binario'].sum()/df_acao.shape[0]

In [ ]:
df = df_acao.copy()

df = df.drop(columns = ['proximo_candle','stop','tp','TRIX'])
df = df.dropna()
inicio = '2022-11-01'
termino = '2022-01-01'

df_train = df[(df['Data'] > '2022-01-01') & (df['Data'] < inicio)]
df_train = df_train.set_index(['Data','acao'])

df_test = df[(df['Data'] > inicio)]
df_test = df_test.set_index(['Data','acao'])

y_train = df_train['resultado_binario']
x_train = df_train.drop(columns=['resultado_binario'])

y_test = df_test['resultado_binario']
x_test = df_test.drop(columns=['resultado_binario'])

In [ ]:
df_test['resultado_binario'].sum()/df_test.shape[0]

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [ ]:
#clf = ExtraTreesClassifier()
#clf = LogisticRegression()
clf = lgb.LGBMClassifier()
clf.fit(x_train, y_train)

# predict the results
y_pred=clf.predict(x_test)
y_proba = clf.predict_proba(x_test)[:,1]
# view accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

print('Training set score: {:.4f}'.format(clf.score(x_train, y_train)))

print('Test set score: {:.4f}'.format(clf.score(x_test, y_test)))

In [ ]:
y_pred.sum()

In [ ]:
x_test = x_test.reset_index()
x_test['predito'] = y_pred
x_test['resultado_binario'] = y_test.values
x_test['proba'] = y_proba

In [ ]:
new = x_test[['Data','acao','resultado_binario','predito']]
new = x_test[(x_test['predito'] == 1) & ((x_test['proba'] >= 0.5))]
new['certo'] = new.apply(lambda x: 1 if x['resultado_binario'] == x['predito'] else 0, axis = 1)

In [ ]:
#new[['Data','acao','resultado_binario','predito','certo']]

In [ ]:
new['certo'].sum()/new.shape[0]

In [ ]:
new.shape

In [ ]:
new